<img src="PEST++V3_cover.jpeg" style="float: left">

<img src="flopylogo.png" style="float: right">

<img src="AW&H2015.png" style="float: center">

# History match the Freyberg model adding a ``RCH`` parameter to our ``K`` parameter

### Remember that when something is not included in the uncertainty analysis it is the same as saying that we know it perfectly.  Recharge is hard to measure accurately in the field, and difficult to upscale to a model domain.  So, we clearly don't know it perfectly and, therefore, typically include it in the parameter estimation process.  That's what we'll do in this exercise. 


# Quick reminder of what the model looks like:

It is a heterogenous 2D areal (1-layer) model that is a step up in complexity from our xsec model. Recall it looks like this, as shown in the original Freyberg (1988) paper on the left, and a Groundwater Vistas version on the right (from the file in the GW_Vistas subdirectory).   

<img src="Freyburg1988_fig1.png" style="float: left">

<img src="Freyberg_k_plot_GW_Vistas.png" style="float: right">

### Standard two code blocks to set the notebook up

In [ ]:
%matplotlib inline
import os
import sys
sys.path.append("..")
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import pyemu

In [ ]:
import freyberg_setup as fs
pst_name = fs.PST_NAME_KR
working_dir = fs.WORKING_DIR_KR
fs.setup_pest_kr()
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.control_data.noptmax = 0
pst.observation_data.loc[pst.observation_data.obgnme=='calflux', 'weight']=0
pst.write(os.path.join(working_dir,pst_name))

In [ ]:
fs.plot_model(working_dir, pst_name)

### Let's run PESTCHEK and see what it says about our freyberg.pst file

In [ ]:
pyemu.helpers.run("pestchek {0}".format(pst_name),cwd=working_dir)

### Curious, in the PESTCHEK warning section it says "All parameters belonging to the parameter group "rch" are either fixed or tied". That is flagged as a warning because PESTCHEK is wondering why would it not be adjustable after you went to all the trouble to define it as a parameter.  But, there  are times you may want to do this, so it is classified as a warning and isn't going to stop you.

## But that is not what we want, we want to make recharge a parameter in this activity.

### Open the PEST control file freyberg.pst in your text editor.  

1) Look in the parameter data section

2) Find the parameter __rch_0__ (the recharge for the calibration period) and make it adjustable (hint:  look at the other parameters) 

3) Save the file

4) Run PESTCHEK on the PEST control file in a seperate terminal window or by executing the next code block and looking at the terminal window where you launched this notebook

In [ ]:
pyemu.helpers.run("pestchek {0}".format(pst_name),cwd=working_dir)

Did that parameter group "rch" warning go away?

FYI - You can also change that PEST control file setting with pyemu...

In [ ]:
pst.parameter_data.loc["rch_0","partrans"] = "log"
pst.write(os.path.join(working_dir,pst_name))


4) When no errors, run PEST++ by executing the next block (look in your terminal window to see the run progress and wait for the 0 to show up below the code block before continuing).


In [ ]:
pyemu.helpers.run("pestpp {0}".format(pst_name),cwd=working_dir)

### ``PEST++`` only ran the model one time - why?

#### This was of course in the warnings when we ran PESTCHEK. Let's do that again in the next block and pay attention to that part of the warnings section (look at the terminal window where you launched this notebook to see the PESTCHEK output). 

In [ ]:
pyemu.helpers.run("pestchek {0}".format(pst_name),cwd=working_dir)

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.control_data.noptmax = 20
pst.write(os.path.join(working_dir,pst_name))

#### Let's run PESTCHEK again to make sure

In [ ]:
pyemu.helpers.run("pestchek {0}".format(pst_name),cwd=working_dir)

### Now that we've changed NOPTMAX, let's run a PEST++ again.  We set NOPTMAX to 20 so the run is longer - watch your terminal window for progress.  Again, don't advance until you see a 0 returned below the code block.


#### As you watch your terminal window scroll by, look at the right edge where it reports the counter of the runs needed for each PEST++ iteration.  Why does it change?

In [ ]:
pyemu.helpers.run("pestpp {0}".format(pst_name),cwd=working_dir)

### Let's see how we did for our fit:

In [ ]:
df_obj = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".iobj")),index_col=0)
df_obj

(you can see this information unformatted by opening the .iobj file, which is a comma delimited ASCII PEST++ output file)



### Additional knobs given to PEST gives it more flexibility for fitting the observed data - is the final Phi lower than the last activity?


### You can see individual residuals (and the weights you specified) in the freyberg.rei file.  Here's we'll use Python to calculate the summary statistics for this new run:

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.phi

In [ ]:
pst.phi_components

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
pyemu.plot_utils.res_1to1(pst)

### Hmm, looks familiar.....let's look at the parameter uncertainties:

In [ ]:
df_paru = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".par.usum.csv")),index_col=0)
df_paru

### Hmm - Recharge = 0.00000?  No, it's not 0.0 - recall that when we log parameters the parameter and its uncertainty is reported as logarithms.  So, 10^0 = 1, which is what we see for an initial value in the PEST control file. 

Now let's plot the parameter uncertainty, where the pre-calibration prior is gray dashed and the after-calibration posterior is in blue...

In [ ]:
pyemu.helpers.plot_summary_distributions(df_paru,label_post=True)

### Now let's compare this to the parameter uncertainty from the K-only calibration done previously :

#### Here's the K only case from the last activity that you can compare to the similar K+R table above:

In [ ]:
df_paru_single = pd.read_csv(os.path.join("..","freyberg_k","freyberg_un","freyberg_un.par.usum.csv"),index_col=0)
df_paru_single

#### Easier to see if we plot them. Here's the posterior and prior standard deviation for the new K+R (left) next the to K only results (right)

In [ ]:
df_paru_concat = pd.concat([df_paru,df_paru_single],join="outer",axis=1,keys=["k+r","k_only"])
df_paru_concat.sort_index(inplace=True,axis=1)
#df_paru_concat.sort_index(level=1,inplace=True)

for pname in df_paru_concat.index:
    ax = df_paru_concat.loc[pname,(slice(None),("prior_stdev","post_stdev"))].plot(kind="bar")
    ax.set_title(pname)
    plt.show()

### How does the uncertainty reduction for ``hk`` change when ``rch_0`` is included?

## Now let's look at the forecasts:

#### First, here's our previous results for the K only case:

In [ ]:
df_predu_single = pd.read_csv(os.path.join("..","freyberg_k","freyberg_un","freyberg_un.pred.usum.csv"),index_col=0)
df_predu_single.loc[:,"reduction"] = 100.0 *  (1.0 - (df_predu_single.post_stdev / df_predu_single.prior_stdev))

figs, axes = pyemu.helpers.plot_summary_distributions(df_predu_single,subplots=True)

#### Wow - tight and certain.  Here's this K-only run (now in green) compared to our new run where K and R are parameters (blue):

In [ ]:
df_predu = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".pred.usum.csv")),index_col=0)
df_predu.loc[:,"reduction"] = 100.0 *  (1.0 - (df_predu.post_stdev / df_predu.prior_stdev))

for forecast in df_predu.index:
    ax = plt.subplot(111)
    pyemu.helpers.plot_summary_distributions(df_predu.loc[[forecast],:],ax=ax)
    pyemu.helpers.plot_summary_distributions(df_predu_single.loc[[forecast],:],ax=ax,pt_color='g')
    ax.set_title(forecast)
    ax.grid()
    plt.show()

### Which forecasts are influenced by the ``rch_0`` parameter?

### The higher and narrower the peak the more certain we are - did the uncertainty decrease or increase when we added the second parameter?

### Which forecasts had little change - why?

### Which case (``K`` or ``K+R``) provides the more robust uncertainty estimate?

# Wait, something is amiss.  

### Look at this slightly modified version of the groundwater governing equation from Anderson et al. (2015) below.  Is this problem well posed? That is, if recharge increased (represented by an increase in $W^*$) *and* at the same time K increased (represented by an increase in q) could they offset each other so that the righthand side stays the same? What is this called?

 
  <img src="GW_GE2.jpg" style="float: center">
 
 
 ### Recall the Bravo et al. (2002) trough when calibrating K and R with only heads:
 
  <img src="Fig9.11a_bravo_trough.jpeg" style="float: center">
  
  ****************
 

# Let's look at the correlation of our two parameters

In [ ]:
sc = pyemu.Schur(os.path.join(working_dir,pst_name.replace(".pst",".jcb")))
cov = pyemu.Cov(sc.xtqx.x, names=sc.pst.adj_par_names)
R = cov.to_pearson()
R.df()

### Wow - correlated.  Recall that Hill and Tiedeman (2007) suggest that correlation > 0.95 or so is not estimable

## Even though estimating both R and K using only head observations is not possible because of this correlation, PEST++ gave you an answer.  How? 

(Hint:  look in the .rec file and see what happens to the parameters over the course of the run)


## Let's dig into the PEST result a bit more...

1) Compare the optimal rch1 parameter value in the __freyberg_kr.rec__ file (search for "Optimal parameter values" without the quotation marks) to the rch_0 parameter data the instructors supplied in the PEST control file.  

2) Where does the optimal parameter lie with respect to the bounds that were given for the parameter? 

3) Open the __freyberg_kr.ipar__ file in a text editor.  What was the iteration history of rch_0?

## So how did PEST++ deal with this intractable correlation?

# Last points:

## Do you believe that value is optimal, or even defensible?  Should we believe the forecast uncertainty either?

Let's again look at the forecasts with the "Truth"

In [ ]:
figs, axes = pyemu.helpers.plot_summary_distributions(df_predu,subplots=True)
for ax in axes:
    fname = ax.get_title().lower()
    ylim = ax.get_ylim()
    v = pst.observation_data.loc[fname,"obsval"]
    ax.plot([v,v],ylim,"k--")
    ax.set_ylim(ylim)

Still not bracketing the "truth" for many forecasts...yikes!

# Advanced

### Let's test how "optimal" these parameters are. 

1) Open freyberg_kr\freyberg_kr.pst in a text editor, 

2) change the upper bound of the rch_0 parameter from 1.25 to 3.25

3) re-run pestpp

4) open the __freyberg_kr.ipar__ file in a text editor.  

How does the new optimal rch_0 compare to the previous optimal value? To the new upper bound?